In [6]:
import pandas as pd
import numpy as np
import geoenrich
from geoenrich import dataloader, enrichment, satellite, exports
import pathlib

pd.set_option('display.max_rows', 200)

In [7]:
df = pd.read_csv('../plankton_geoenrich_data/biodiv/plankton_med.csv')

In [3]:
variables = [
    'sst',
    'nh4_med',
    'no3_med',
    'po4_med',
    'o2_med',
    'chl_med',
    'thetao_med',
    'so_med',
    ]

In [4]:
df.shape

(62275, 96)

In [4]:
# output = exports.retrieve_data("plankton", 3897, 'oxygen', shape = 'buffer')
# values = exports.export_to_array(output)

# exports.collate_npy(ds_ref = 'plankton_med', data_path = './npy', slice = (0, 10000))

In [19]:
data_path + '/' + 'plankton.csv'

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'